In [2]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        return sum(series)/len(series)
    return averager

In [3]:
avg = make_averager()

In [4]:
avg

<function __main__.make_averager.<locals>.averager(new_value)>

In [5]:
avg(10)

10.0

In [6]:
avg(11)

10.5

In [7]:
avg.__code__

<code object averager at 0x11206b8a0, file "<ipython-input-2-ba1550cee893>", line 3>

In [8]:
dir(avg.__code__)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_stacksize',
 'co_varnames']

In [9]:
# closure is a function that retains the bindings of the free variables
# that exist when the function is defined, so that they can be used later
# when the function is invoked and the defining scope is no longer available
avg.__closure__

(<cell at 0x1120a0c10: list object at 0x11207a410>,)

In [14]:
# nonlocal: flag a variable as a free variable even when it is assigned
# a new value within the function

def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [15]:
avg = make_averager()

In [16]:
avg(10)

10.0

In [17]:
import time

# 1. does not support keyword arguments
# 2. marks the __name__ and __doc__ of the decorated function
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [18]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [19]:
snooze(10)

[10.00505666s] snooze(10) -> None


In [21]:
import time
from functools import wraps

def clock(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return wrapper

In [24]:
@clock
def tag(name, *content, cls=None, **attrs):
    '''generate one or more HTML tags'''
    if cls is not None:
        attrs['class'] = cls
        
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
        
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                        (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [26]:
tag('p', 'name', cls='sidebar', content='testing')

[0.00001068s] tag('p', 'name', cls='sidebar', content='testing') -> '<p class="sidebar" content="testing">name</p>'


'<p class="sidebar" content="testing">name</p>'

In [31]:
from functools import lru_cache

@lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

for i in range(5):
    fibonacci(i + 1)

[0.00000084s] fibonacci(1) -> 1
[0.00000042s] fibonacci(0) -> 0
[0.00002616s] fibonacci(2) -> 1
[0.00000083s] fibonacci(3) -> 2
[0.00000062s] fibonacci(4) -> 3
[0.00000059s] fibonacci(5) -> 5


In [32]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)


@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)


@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)


@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'


In [33]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  '__name__',
  'def make_averager():\n    series = []\n    def averager(new_value):\n        series.append(new_value)\n        return sum(series)/len(series)\n    return averager',
  'avg = make_averager()',
  'avg',
  'avg(10)',
  'avg(11)',
  'avg.__code__',
  'dir(avg.__code__)',
  'avg.__closure__',
  '# nonlocal: flag a variable as a free variable even when it is assigned\n# a new value within the function\n\ndef make_averager():\n    count = 0\n    total = 0\n    def averager(new_value):\n        nonlocal count, total\n        count += 1\n        total += new_value\n        return total / count',
  'avg = make_averager()\navg(10)',
  'avg = make_averager()',
  'avg',
  '# nonlocal: flag a variable as a free

In [36]:
def trylocals(*args, **kwargs):
    a = 1
    print(locals())
    
trylocals("a", 1, [2], {3: 4}, key='value')

{'args': ('a', 1, [2], {3: 4}), 'kwargs': {'key': 'value'}, 'a': 1}
